In [ ]:
import pandas as pd
import scipy.stats as stats
import numpy as np
from statsmodels.stats.multitest import multipletests

# Конфигурация путей
PATHS = {
    'input': {
        'combined_scores': r"../../depmap_datasets/prepared_datasets/depscores.csv",
        'merged_data': r"../../depmap_datasets/prepared_datasets/cancer_cell_lines_merged_data.csv",
        'area_under_curve': r"../../depmap_datasets/prepared_datasets/auc_bosutinib_ctd2.csv"
    },
    'output': {
        'correlation_pim1': r"./pim1_correlation_bosutinib_auc_vs_depscores.csv",
        'correlation_pim2': r"./pim2_correlation_bosutinib_auc_vs_depscores.csv",
        'correlation_pim3': r"./pim3_correlation_bosutinib_auc_vs_depscores.csv"
    }
}

def process_auc_data():
    """Загрузка и обработка AUC данных с удалением дубликатов"""
    print("\nОбработка дубликатов в файле AUC...")
    df_auc = pd.read_csv(PATHS['input']['area_under_curve'])
    
    # Выводим общее количество строк до обработки
    print(f"Всего строк в AUC файле: {len(df_auc)}")
    
    # Находим дубликаты DepMap_ID
    duplicates = df_auc[df_auc.duplicated('ModelID', keep=False)]
    
    if not duplicates.empty:
        # Выводим информацию о дубликатах
        print("\nНайдены следующие дубликаты DepMap_ID:")
        print(duplicates.sort_values(['ModelID', 'area_under_curve']).to_string(index=False))
        
        # Оставляем строку с минимальным AUC для каждого дубликата
        df_auc = df_auc.loc[df_auc.groupby('ModelID')['area_under_curve'].idxmin()]
        
        # Выводим информацию об оставленных строках
        kept_records = df_auc[df_auc['ModelID'].isin(duplicates['ModelID'])]
        print("\nОставлены следующие записи (с минимальным AUC):")
        print(kept_records.sort_values('ModelID').to_string(index=False))
        
        # Выводим статистику
        print(f"\nУдалено дубликатов: {len(duplicates) - len(kept_records)}")
        print(f"Оставлено уникальных записей: {len(kept_records)}")
    else:
        print("Дубликаты DepMap_ID не найдены")
    
    print("Обработка дубликатов завершена.\n")
    return df_auc

def process_data():
    # 1. Загрузка и фильтрация генов PIM
    df = pd.read_csv(PATHS['input']['combined_scores'])
    genes_to_keep = ["PIM1", "PIM2", "PIM3"]
    genes_in_file = [gene for gene in genes_to_keep if gene in df.columns]
    
    if not genes_in_file:
        raise ValueError("Ни один из генов PIM не найден в файле")
    
    df_pim = df[['DepMap_ID'] + genes_in_file]

    # 2. Загрузка данных о заболеваниях
    df_merged = pd.read_csv(PATHS['input']['merged_data'])
    
    df_filtered = pd.merge(
        df_pim,
        df_merged[['ModelID', 'primary_disease_renamed']],
        left_on='DepMap_ID',
        right_on='ModelID',
        how='inner'
    ).drop(columns=['ModelID'])

    # 3. Добавление AUC данных с обработкой дубликатов
    df_auc = process_auc_data()
    df_final = pd.merge(
        df_filtered,
        df_auc[['ModelID', 'area_under_curve']],
        left_on='DepMap_ID',
        right_on='ModelID',
        how='inner'
    ).drop(columns=['ModelID'])

    return df_final, genes_in_file

def calculate_correlations(df_final, genes):
    results = {}
    
    for gene in genes:
        corr_results_spearman = []
        pval_results_spearman = []
        corr_results_pearson = []
        pval_results_pearson = []
        n_points_results = []  # Изменено с n_lines на n_points
        skip_reasons = []
        
        for cancer_type, group in df_final.groupby('primary_disease_renamed'):
            gene_data = group[gene].replace([np.inf, -np.inf], np.nan).dropna()
            auc_data = group['area_under_curve'].replace([np.inf, -np.inf], np.nan).dropna()
            
            # Объединение данных с удалением NA
            valid_data = pd.DataFrame({gene: gene_data, 'auc': auc_data}).dropna()
            n_points = len(valid_data)  # Теперь это число точек, а не линий
            n_points_results.append(n_points)
            
            # Инициализация результатов
            corr_spearman, pval_spearman = np.nan, np.nan
            corr_pearson, pval_pearson = np.nan, np.nan
            skip_reason = None
            
            # Проверка условий для расчета корреляции
            if n_points < 10:
                skip_reason = f"Недостаточно точек ({n_points} < 10)"  # Обновлено сообщение
            elif valid_data[gene].nunique() == 1:
                skip_reason = f"Постоянные значения гена {gene}"
            elif valid_data['auc'].nunique() == 1:
                skip_reason = "Постоянные значения AUC"
            else:
                try:
                    # Расчет Спирмена
                    corr_spearman, pval_spearman = stats.spearmanr(valid_data[gene], valid_data['auc'])
                    # Расчет Пирсона
                    corr_pearson, pval_pearson = stats.pearsonr(valid_data[gene], valid_data['auc'])
                except Exception as e:
                    skip_reason = f"Ошибка расчета: {str(e)}"
            
            if skip_reason:
                print(f"Пропуск {gene} для {cancer_type}: {skip_reason}")
                skip_reasons.append(skip_reason)
            else:
                skip_reasons.append("")
                
            corr_results_spearman.append([cancer_type, corr_spearman])
            pval_results_spearman.append([cancer_type, pval_spearman])
            corr_results_pearson.append([cancer_type, corr_pearson])
            pval_results_pearson.append([cancer_type, pval_pearson])
        
        # Расчет p-adj (поправка Бенджамини-Хохберга)
        pvals_spearman = [x[1] for x in pval_results_spearman if not np.isnan(x[1])]
        _, pvals_adj_spearman, _, _ = multipletests(pvals_spearman, method='fdr_bh')
        
        pvals_pearson = [x[1] for x in pval_results_pearson if not np.isnan(x[1])]
        _, pvals_adj_pearson, _, _ = multipletests(pvals_pearson, method='fdr_bh')
        
        # Создание DataFrame с результатами
        df_result = pd.DataFrame({
            'Cancer_Type': [x[0] for x in corr_results_spearman],
            'N_Points': n_points_results,  # Изменено с N_Lines на N_Points
            f'{gene}_Spearman_Correlation': [x[1] for x in corr_results_spearman],
            f'{gene}_Spearman_p_value': [x[1] for x in pval_results_spearman],
            f'{gene}_Spearman_p_adj': np.nan,
            f'{gene}_Pearson_Correlation': [x[1] for x in corr_results_pearson],
            f'{gene}_Pearson_p_value': [x[1] for x in pval_results_pearson],
            f'{gene}_Pearson_p_adj': np.nan,
            'Skip_Reason': skip_reasons
        })
        
        # Заполнение скорректированных p-значений
        adj_idx = 0
        for i in range(len(df_result)):
            if not np.isnan(df_result.at[i, f'{gene}_Spearman_p_value']):
                df_result.at[i, f'{gene}_Spearman_p_adj'] = pvals_adj_spearman[adj_idx]
                df_result.at[i, f'{gene}_Pearson_p_adj'] = pvals_adj_pearson[adj_idx]
                adj_idx += 1
        
        results[gene] = df_result
    
    return results

def save_results(results):
    for gene, df in results.items():
        output_path = PATHS['output'][f'correlation_{gene.lower()}']
        df.to_csv(output_path, index=False)
        print(f"Результаты для {gene} сохранены: {output_path}")

def main():
    print("Начало обработки данных...")
    
    # Обработка данных
    df_final, genes = process_data()
    
    # Расчет корреляций
    print("\nРасчет корреляций...")
    correlation_results = calculate_correlations(df_final, genes)
    
    # Сохранение результатов
    print("\nСохранение результатов...")
    save_results(correlation_results)
    
    print("\nОбработка успешно завершена!")

if __name__ == "__main__":
    main()

Начало обработки данных...

Обработка дубликатов в файле AUC...
Всего строк в AUC файле: 794

Найдены следующие дубликаты DepMap_ID:
 experiment_id    ModelID  area_under_curve  master_ccl_id ccl_name
            44 ACH-000219           11.5880             30     A375
           421 ACH-000219           12.4660             30     A375
           733 ACH-000222           11.1570             52    ASPC1
           629 ACH-000222           11.6350             52    ASPC1
           483 ACH-000329           13.2340            131 CCFSTTG1
           497 ACH-000329           13.3370            131 CCFSTTG1
           179 ACH-000463           12.8620         128001  NCIH460
           352 ACH-000463           13.2640         128001  NCIH460
           777 ACH-000507           10.2620            526     KE39
           190 ACH-000507           11.2120            526     KE39
            89 ACH-000510           12.2120         155395 NCIH1299
           195 ACH-000510           12.6810        